<a href="https://colab.research.google.com/github/Daniel-R-Armstrong/geo2vec/blob/master/geo2vec_Census_Tract_Housing_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Census Tract Housing Classification

## Census Tract Classification Types


>### **Urban Core**: 
> density > 2,900 sq. km  
>### **Older Suburban**:
> median house built 1946 to 1979,   
> density < 2,900 sq. km and density > 100 sq. km  
>### **Suburban**:
> median house built after 1979,
> density < 2,900 sq. km and density > 100 sq. km  
>### **New Suburban**:
> median house built after 2000,
> density < 2,900 sq. km and density > 100 sq. km
>### **Exurban**: 
>All others  

This Idea of classification is based on the concepts from [newgeography.com](http://www.newgeography.com/content/004349-from-jurisdictional-functional-analysis-urban-cores-suburbs)  and [IBM's Data Lab](https://github.com/ibm-watson-data-lab/open-data/blob/master/urbanity) I amended it to fit my needs. This concept is based on how urban citys grew and the impact of cars, I added a section for newly created surburban areas created during and after the housing boom. 

##Setup

In [ ]:
inputs('ACSDT5Y2017_il_data_2.csv': "a modified version of 2017 ACS")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/'My Drive'/data/geo2vec/il/data/loop_listings

/content/drive/My Drive/data/geo2vec/il/data/loop_listings


In [ ]:
ls

In [ ]:
import pandas as pd

## ACS
(tract 'population' and 'Median year built')

In [ ]:
def num2str(i):
    return str(i)

converters = {'GEOID10' : num2str,
              'state_fips' : num2str,
              'county_fips' : num2str,
              'tract_fips' : num2str,
              'GEOID_ST_CTY_TRT_2010' : num2str,
              'state' : num2str,
              'county' : num2str,
              'tract' : num2str}

In [ ]:
#load in the 2017 acs file 
acs=pd.read_csv('ACSDT5Y2017_il_data_2.csv',index_col=0, converters=converters)
acs.head(5)

,B01001A_001E,B25001_001E,B25035_001E,B25058_001E,B25071_001E,B25092_001E,B17006_001E,B19083_001E,B19058_001E,B19053_001E,B19054_001E,B19052_001E,B08012_033E,B08012_032E,state,county,tract,B08012_035E,B08012_034E,B08012_031E,B08012_030E,B08012_037E,B08012_036E,B08012_039E,B08012_038E,B08012_021E,B08012_020E,B08012_023E,B08012_022E,B08012_029E,B08012_028E,B08012_025E,B08012_024E,B08012_027E,B08012_026E,B08012_011E,B08012_010E,B08012_017E,B08012_016E,B08012_019E,B08012_018E,B08012_013E,B08012_012E,B08012_015E,B08012_014E,B08012_005E,B08012_004E,B08012_007E,B08012_006E,B08012_001E,B08012_003E,B08012_002E,B08012_009E,B08012_008E,GEOID10,state_fips,county_fips,tract_fips,GEOID_ST_CTY_TRT_2010
0,4967,1898,1989,1888,50.0,22.8,1222,0.4145,1853,1853,1853,1853,156,145,17,197,881002,37,182,136,98,117,84,15,86,221,76,147,65,110,0,197,188,1166,43,305,231,88,56,80,167,58,283,8,1336,303,186,232,225,2502,166,8,102,403,17197881002,17,17197,17197881002,17197881002
1,6152,2047,1994,-666666666,-666666666.0,21.5,1497,0.3541,2011,2011,2011,2011,121,171,17,197,881105,45,183,197,105,272,79,25,66,302,63,77,103,185,41,349,224,1490,34,496,156,157,80,227,241,59,415,11,1868,438,262,184,398,3358,265,52,148,485,17197881105,17,17197,17197881105,17197881105
2,5062,1875,1975,1169,34.8,17.8,1277,0.3301,1698,1698,1698,1698,43,78,17,197,880502,19,130,275,140,100,56,71,178,203,18,84,121,314,62,242,202,1466,15,302,140,37,122,258,104,86,420,27,1433,379,177,61,336,2899,436,89,140,333,17197880502,17,17197,17197880502,17197880502
3,2609,1579,1963,619,43.0,17.9,765,0.4289,1342,1342,1342,1342,91,54,17,197,883700,13,76,141,96,61,19,19,23,81,75,15,34,20,24,46,27,637,18,88,34,120,70,127,73,37,69,24,710,214,216,166,181,1347,90,48,47,157,17197883700,17,17197,17197883700,17197883700
4,3999,2385,1981,1178,38.2,22.6,2012,0.3197,2247,2247,2247,2247,41,154,17,197,880105,58,398,197,129,175,213,17,158,449,162,66,117,168,97,318,275,1805,39,450,279,285,213,173,256,56,476,44,2397,453,414,203,327,4202,381,141,175,847,17197880105,17,17197,17197880105,17197880105


In [ ]:
acs = acs.set_index('GEOID_ST_CTY_TRT_2010') #state, County, tract, 2010 vintage
acs['population'] = acs.B01001A_001E
acs['median_year_built']=acs.B25035_001E

## TIGER tract shape file 
(tract land area)

TIGER/Line Shapefiles and Geodatabases pre-joined with 2010 Census select Demographic Profile data, which is a nice way you explore and visilulaize your data, the biggest problem is that it is now almost 10 years old. Since the tracts dont change it is ok to us them and combine it with more recent demograpic data from other sources. 

In [ ]:
#!wget https://www2.census.gov/geo/tiger/TIGER2010DP1/Tract_2010Census_DP1.zip  -nv

In [ ]:
# # Unzip
# import zipfile
# zf = zipfile.ZipFile('Tract_2010Census_DP1.zip')
# zf.extractall()

In [ ]:
!pip install geopandas -q

     |████████████████████████████████| 901kB 3.3MB/s 
     |████████████████████████████████| 17.9MB 40.1MB/s 
     |████████████████████████████████| 11.2MB 41.4MB/s 


In [ ]:
import geopandas as gpd

In [ ]:
#Tract_2010Census_DP1.shp:
# load tracts shapefile that contains pop data
tiger_2010 = gpd.read_file('Tract_2010Census_DP1.shp',converters=converters) 
len(tiger_2010)

74002

In [ ]:
tiger_2010.AREA_M_SQ

In [ ]:
tiger_2010.rename(columns={'ALAND10' : 'AREA_M_SQ', 'GEOID10' : 'GEOID_ST_CTY_TRT_2010'},inplace=True)

In [ ]:
tiger_2010.to_pickle('tiger_2010')

### optional filter (just chicago land)

In [ ]:
Dont use if you are using multiple states


# filter to retain only tracts in chicago

state = '17'
counties = ['031','037','043', '063', '089', '093','097', '111', '197']
state_counties = ['{}{}'.format(state, county) for county in counties]
tiger_2010['state_county'] = tiger_2010['GEOID10'].map(lambda x: x[0:5])


#17031. Cook County, IL (5,236,393)
#17037. DeKalb County, IL (104,345)
#17043. DuPage County, IL (930,412)
#17063. Grundy County, IL (50,277)
#17089. Kane County, IL (524,886)
#17093. Kendall County, IL (120,036)
#17097. Lake County, IL (702,898)
#17111. McHenry County, IL (307,357)
#17197. Will County, IL (683,995)

In [ ]:
tiger_2010.head(3)

,GEOID10,NAMELSAD10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,DP0010001,DP0010002,DP0010003,DP0010004,DP0010005,DP0010006,DP0010007,DP0010008,DP0010009,DP0010010,DP0010011,DP0010012,DP0010013,DP0010014,DP0010015,DP0010016,DP0010017,DP0010018,DP0010019,DP0010020,DP0010021,DP0010022,DP0010023,DP0010024,DP0010025,DP0010026,DP0010027,DP0010028,DP0010029,DP0010030,DP0010031,DP0010032,DP0010033,DP0010034,...,DP0130001,DP0130002,DP0130003,DP0130004,DP0130005,DP0130006,DP0130007,DP0130008,DP0130009,DP0130010,DP0130011,DP0130012,DP0130013,DP0130014,DP0130015,DP0140001,DP0150001,DP0160001,DP0170001,DP0180001,DP0180002,DP0180003,DP0180004,DP0180005,DP0180006,DP0180007,DP0180008,DP0180009,DP0190001,DP0200001,DP0210001,DP0210002,DP0210003,DP0220001,DP0220002,DP0230001,DP0230002,Shape_Leng,Shape_Area,geometry
0,01005950300,Census Tract 9503,352811329.0,499970.0,+31.7908933,-085.5670514,1638,98,87,88,115,76,96,78,94,90,91,129,118,157,87,73,62,61,38,774,43,45,37,61,33,50,41,45,42,38,68,53,75,35,...,673,462,148,331,98,36,12,95,38,211,192,84,35,108,61,193,237,2.43,2.97,832,673,159,29,1,9,0,31,89,1.6,18.2,673,544,129,1283,355,2.36,2.75,1.164633,0.033648,POLYGON ((-85.52743699999996 31.86650300000008...
1,01005950900,Census Tract 9509,16201446.0,8339342.0,+31.8467221,-085.1462332,4583,456,387,363,348,326,313,230,229,274,300,291,265,227,169,148,114,91,52,2057,222,204,180,182,140,128,112,91,112,115,128,129,98,68,...,1784,1215,611,624,261,78,32,513,318,569,529,193,57,336,166,728,476,2.57,3.18,1953,1784,169,54,0,14,1,41,59,1.6,5.5,1784,857,927,2196,2387,2.56,2.57,0.280687,0.002338,POLYGON ((-85.16412199999996 31.83059499999996...
2,01005950800,Census Tract 9508,14630162.0,380622.0,+31.9206930,-085.1760317,2055,110,137,128,150,108,112,104,111,115,148,145,148,140,127,76,64,81,51,938,55,64,68,78,63,42,52,51,53,71,55,65,52,61,...,901,588,235,381,114,35,17,172,104,313,283,87,28,196,99,276,298,2.28,2.85,1008,901,107,10,1,19,1,39,37,3.3,2.8,901,550,351,1257,798,2.29,2.27,0.173100,0.001431,"POLYGON ((-85.14871799999997 31.909335, -85.14..."


In [ ]:
# The GEOID is not always the same so I use GEOID_ST_CTY_TRT_2010 so I know it meets my standard (len=11,state=2, county=3, tract=6)
tiger_2010.rename(columns={'GEOID10':'GEOID_ST_CTY_TRT_2010','ALAND10': 'AREA_M_SQ'},inplace=True) 

### Area Subset

In [ ]:
# # retain only subset of columns then index by tract fips code
# tracts = tracts_chicago[['geometry', 'ALAND10', 'DP0010001']]
# tracts.index = tracts_chicago['GEOID10']
# tracts.head(3)

#use all
tiger_tracts = tiger_2010[['GEOID_ST_CTY_TRT_2010','geometry', 'AREA_M_SQ']]
tiger_tracts.index = tiger_tracts['GEOID_ST_CTY_TRT_2010']
#tracts.head(3)
len(tiger_tracts)

74002

In [ ]:
tiger_tracts.to_pickle('tiger_tracts_GEO_Area_2010')

In [ ]:
ls

## Merge 
(acs and tiger)

In [ ]:
# merge acs and tiger
acs_tiger = pd.merge(acs, tiger_tracts, how='left', left_index=True, right_index=True) 
len(acs_tiger)

3123

In [ ]:
# acs_tiger.head(3)

### Population Density
(persons per square kilometer)

persons per square km = persons / (area in square meters / 1,000,000)  
persons per hectare = persons / (area in square meters / 10,000)  

In [ ]:
acs_tiger['POP_PER_KM_SQ'] = acs_tiger['population'] / (acs_tiger['AREA_M_SQ']/1000000)
acs_tiger.sample(4)

,B01001A_001E,B25001_001E,B25035_001E,B25058_001E,B25071_001E,B25092_001E,B17006_001E,B19083_001E,B19058_001E,B19053_001E,B19054_001E,B19052_001E,B08012_033E,B08012_032E,state,county,tract,B08012_035E,B08012_034E,B08012_031E,B08012_030E,B08012_037E,B08012_036E,B08012_039E,B08012_038E,B08012_021E,B08012_020E,B08012_023E,B08012_022E,B08012_029E,B08012_028E,B08012_025E,B08012_024E,B08012_027E,B08012_026E,B08012_011E,B08012_010E,B08012_017E,B08012_016E,B08012_019E,B08012_018E,B08012_013E,B08012_012E,B08012_015E,B08012_014E,B08012_005E,B08012_004E,B08012_007E,B08012_006E,B08012_001E,B08012_003E,B08012_002E,B08012_009E,B08012_008E,GEOID10,state_fips,county_fips,tract_fips,population,median_year_built,GEOID_ST_CTY_TRT_2010,geometry,AREA_M_SQ,POP_PER_KM_SQ
GEOID_ST_CTY_TRT_2010,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
17031803016,1317,950,1974,1398,19.9,18.2,684,0.3655,899,899,899,899,25,80,17,31,803016,19,51,54,36,41,54,25,37,132,61,28,10,50,5,79,34,477,39,75,82,53,18,199,65,64,116,0,718,119,89,86,279,1195,68,5,29,183,17031803016,17,17031,17031803016,1317,1974,17031803016,POLYGON ((-87.99751199999997 42.12437799999998...,1029964.0,1278.685469
17091011000,347,1053,1979,566,44.0,21.0,570,0.5072,808,808,808,808,56,22,17,91,11000,15,49,19,45,17,20,36,4,89,12,16,0,14,25,9,30,322,20,47,36,8,21,53,19,56,13,5,282,38,53,68,75,604,35,30,15,138,17091011000,17,17091,17091011000,347,1979,17091011000,POLYGON ((-87.52634399999999 41.04314600000004...,143639350.0,2.415773
17027900500,5679,2392,1972,432,27.0,15.0,1030,0.4357,2109,2109,2109,2109,74,118,17,27,900500,13,93,162,256,38,20,4,41,76,97,39,21,171,63,35,123,1053,31,161,59,268,148,90,139,35,76,71,1138,301,524,171,208,2191,319,134,34,169,17027900500,17,17027,17027900500,5679,1972,17027900500,POLYGON ((-89.14275299999991 38.53872800000005...,181159675.0,31.348036
17061973700,1775,954,1948,394,26.9,17.2,487,0.4391,701,701,701,701,43,15,17,61,973700,0,52,24,49,15,10,0,16,42,42,0,4,64,28,28,23,316,7,38,10,42,81,46,6,7,44,7,328,30,91,85,61,644,145,35,4,94,17061973700,17,17061,17061973700,1775,1948,17061973700,"POLYGON ((-90.3794749999999 39.47523400000006,...",11599105.0,153.029048


### Housing Classification 


In [ ]:
def classification(df):
    if (df['POP_PER_KM_SQ'] >= 2900):
        return 'URBAN CORE'
    elif (df['POP_PER_KM_SQ'] < float(2900) and df['POP_PER_KM_SQ'] >= float(100) and df['median_year_built'] >= float(2000)): 
        return 'NEW SUBURBAN'
    elif (df['POP_PER_KM_SQ'] < float(2900) and df['POP_PER_KM_SQ'] >= float(100) and df['median_year_built'] >= float(1980)): 
        return 'SUBURBAN'
    elif (df['POP_PER_KM_SQ'] < float(2900) and df['POP_PER_KM_SQ'] >= float(100) and df['median_year_built'] < float(1980) and df['median_year_built'] >= float(1946)):  
        return 'OLDER SUBURBAN' 
    else:
        return 'EXURBAN'

In [ ]:
acs_tiger['classification'] = acs_tiger.apply(classification, axis=1)

In [ ]:
#acs_tiger.head(10)

In [ ]:
acs_tiger.classification.describe()

count               3123
unique                 4
top       SUBURBAN_EARLY
freq                1171
Name: CAT, dtype: object